In [3]:
import re
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv("intrinsic.csv")


In [5]:
# Obtener los valores de la columna 'text'
textos = df["text"].values.astype('U')

# Crear una instancia de TfidfVectorizer
vectorizador = TfidfVectorizer(stop_words="english")

# Ajustar el vectorizador a los textos y transformar los textos en una matriz TF-IDF
matriz_tfidf = vectorizador.fit_transform(textos)

# Obtener las palabras únicas (características)
palabras = vectorizador.get_feature_names_out()

# Obtener las frecuencias TF-IDF de las palabras en todos los documentos
frecuencias_tfidf = np.array(matriz_tfidf.sum(axis=0)).flatten()

# Crear un DataFrame para almacenar las palabras y sus frecuencias TF-IDF
df_frecuencias = pd.DataFrame({"Palabra": palabras, "Frecuencia TF-IDF": frecuencias_tfidf})

# Ordenar el DataFrame por frecuencia TF-IDF en orden descendente
df_frecuencias_ordenadas = df_frecuencias.sort_values(by="Frecuencia TF-IDF", ascending=False)


# Obtener las 100 palabras más frecuentes y las 100 palabras menos frecuentes
palabras_mas_frecuentes = df_frecuencias_ordenadas.head(100)
palabras_menos_frecuentes = df_frecuencias_ordenadas.tail(100)

In [6]:
# Configurar pandas para mostrar el DataFrame completo de 100 filas
pd.set_option("display.max_rows", 100)

# Imprimir las palabras más frecuentes
print("Palabras más frecuentes:")
print(palabras_mas_frecuentes)

# Imprimir las palabras menos frecuentes
print("\nPalabras menos frecuentes:")
print(palabras_menos_frecuentes)

Palabras más frecuentes:
             Palabra  Frecuencia TF-IDF
5964         sherman           0.581146
2917         general           0.549349
7214           women           0.454348
4185             men           0.392586
7211           woman           0.317140
6221           state           0.304551
6641            time           0.269459
6604          thomas           0.251830
4580           order           0.241637
3837             law           0.230773
4599         orleans           0.201836
1465      convention           0.201836
3230            hood           0.198558
556          atlanta           0.188872
4415             new           0.174894
7167            wife           0.173190
1495           corps           0.171135
3296         husband           0.170941
454             army           0.169876
2024        district           0.166779
2975             god           0.159695
4062             man           0.158546
5039       president           0.154329
4226        mil

In [7]:
# Configurar pandas para mostrar el DataFrame completo de 100 filas
pd.set_option("display.max_rows", 100)

# Imprimir las palabras más frecuentes
print("Palabras más frecuentes:")
print(palabras_mas_frecuentes)

# Imprimir las palabras menos frecuentes
print("\nPalabras menos frecuentes:")
print(palabras_menos_frecuentes)

Palabras más frecuentes:
             Palabra  Frecuencia TF-IDF
5964         sherman           0.581146
2917         general           0.549349
7214           women           0.454348
4185             men           0.392586
7211           woman           0.317140
6221           state           0.304551
6641            time           0.269459
6604          thomas           0.251830
4580           order           0.241637
3837             law           0.230773
4599         orleans           0.201836
1465      convention           0.201836
3230            hood           0.198558
556          atlanta           0.188872
4415             new           0.174894
7167            wife           0.173190
1495           corps           0.171135
3296         husband           0.170941
454             army           0.169876
2024        district           0.166779
2975             god           0.159695
4062             man           0.158546
5039       president           0.154329
4226        mil

Es interesante tener en cuenta que palabras como woman man, state, time que aparecen entre las palabras mas frecuentes con el df limpio aparecen aquí entre las palabras mas frecuentes también, con un valor de TF IDF inferior a 0.4

Sería interesante profundizar el análisis estableciendo un umbral mínimo de tf idf para desde allí determinar que palabras serían eliminadas

In [11]:
palabras_mas_frecuentes

,Palabra,Frecuencia TF-IDF
5964,sherman,0.581146
2917,general,0.549349
7214,women,0.454348
4185,men,0.392586
7211,woman,0.317140
6221,state,0.304551
6641,time,0.269459
6604,thomas,0.251830
4580,order,0.241637
3837,law,0.230773


In [10]:
one = palabras_mas_frecuentes.loc[palabras_mas_frecuentes['Palabra'] == 'one']
one
#palabras_mas_frecuentes[palabras_mas_frecuentes["Palabra"] == "one"]

,Palabra,Frecuencia TF-IDF


Palabras como one que ocupan los primeros lugares en las palabras mas frecuentes en el ejercicio anterior, ni siquiera ocupan un lugar entre las 100 palabras mas frecuentes ordenadas por valor de TF IDF esto quiere decir que no representaría una utilidad para poder analizar palabras características de un documento.

Creemos que el uso del algorítmo de TF IDF representa un gran interes y utilidad a la hora de identificar palabras que pueden llegar a ser características de un autor, las muletillas o modismos suelen ser aspectos comunes para identificar la escritura de un autor, y el algoritmo nos permitiría identificar rápidamente estas palabras que son características de un documento ponderandolas positivamente. Resulta de interes continuar profundizando.